In [1]:
# import libraries
import numpy as np
import pandas as pd
import logging
import json
from typing import Union, List

In [2]:
# define functions

def read_my_data(filepath: str, low_memory: bool) -> Union[pd.DataFrame, None]:
    """
    Use pandas to read a csv file located at filepath.

    filepath: the location of the csv file.
    low_memory: a boolean flag used to control the memory usage when reading large CSV files.
    return: a pandas DataFrame object, or None if an error occurs.
    """
    try:
        logging.info("Reading the file located at {}".format(filepath))
        df = pd.read_csv(filepath, low_memory=low_memory)
        return df
    except FileNotFoundError as e:
        logging.error("File not found at {}".format(filepath))
    except Exception as e:
        logging.error("An error occurred while reading the file: {}".format(str(e)))
    return None

def get_unique_values(data: pd.DataFrame, column: str) -> Union[int, None]:
    """
    Calculate the number of uniques values in a pandas DataFrame column.

    data: pandas DataFrame as input.
    column: the name of a column.
    return: the number of unique values, or None if an error occurs.
    """
    try:
        logging.info("Calculating the unique values for {}.".format(column))
        unique_values = data[column].nunique()
        return unique_values
    except KeyError:
        logging.error("Column {} not found in the DataFrame.".format(column))
    except Exception as e:
        logging.error("An error occurred while calculating unique values for column {} : {}".format(column, str(e)))
    return None

def calculate_average_value(data: pd.DataFrame, column: str) -> Union[float, None]:
    """
    Calculate the mean value of a numeric column, rounded to two digits.

    data: pandas DataFrame as input.
    column: name of the column.
    return: the average value of the column, or None if an error occurs.
    """
    try:
        if pd.api.types.is_numeric_dtype(data[column]):
            logging.info("Calculating the mean value for column: {}".format(column))
            average_rating = np.mean(data[column])
            rounded_rating = round(average_rating, 2)
            return rounded_rating
        else:
            raise ValueError("Column {} is not numeric.".format(column))
    except KeyError as e:
        logging.error("Column {} not found in DataFrame.".format(column))
    except Exception as e:
        logging.error("An error occurred while calculating the average value for column {} : {}".format(column, str(e)))
    return None

def merge_two_dataframes(data_1: pd.DataFrame,
                         data_2: pd.DataFrame,
                         how: str, on: str) -> Union[pd.DataFrame, None]:
    """
    Merge two pandas dataframes by specifying the join operation and the column used for the join.

    data_1: the first pandas DataFrame.
    data_2: the second pandas DataFrame.
    how: possible variations are: ['inner', 'outer', 'left', 'right']
    on: name of the column used to join the DataFrames.
    return: a merged pandas DataFrame, or None if an error occurs.
    """
    try:
        logging.info('Merging dataframes using {} operation on {} column.'.format(how, on))
        merged_df = pd.merge(data_1, data_2, how=how, on=on)
        return merged_df
    except KeyError as e:
        logging.error("Column {} not found in one of the DataFrames.".format(on))
    except Exception as e:
        logging.error("An error occurred while merging the DataFrames: {}.".format(str(e)))
    return None

def groupby_and_sort(data: pd.DataFrame,
                     groupby_column: str,
                     column: str) -> Union[pd.DataFrame, None]:
    """
    Groupby operation on a specified column, followed by sorting operation.

    data: pandas DataFrame.
    groupby_column: the column on which groupby operation will be performed.
    column: the column on which an average value will be calculated, the values must be numeric.
    return: pandas DataFrame, or None if an error occurs.
    """
    try:
        if pd.api.types.is_numeric_dtype(data[column]):
            logging.info("Performing a GroupBy operation on {} and calculating average on column {}.".format(groupby_column, column))
            average_column_value = data.groupby(groupby_column)[column].mean()
            sorted_df = average_column_value.sort_values(ascending=False)
            return sorted_df
        else:
            raise ValueError("Column '{}' is not numeric.".format(column))
    except KeyError as e:
        logging.error("Column {} not found in DataFrame.".format(column))
    except Exception as e:
        logging.error("An error occurred while performing GroupBy and sorting: {}.".format(str(e)))
    return None

def extract_from_json(genre_list_str: str) -> Union[list, None]:
    """
    Extract movie genres from a JSON-formatted string.

    genre_list_str: A string representing a JSON list containing dictionaries with genre information.
    return: A list of movie genres, or None if an error occurs.
    """
    try:
        logging.info("Extracting genres from JSON string")

        # Replace single quotes with double quotes to ensure JSON compatibility.
        genre_list_str = genre_list_str.replace("'", '"')

        # Load JSON string into a Python list of dictionaries
        genre_list = json.loads(genre_list_str)

        # Extract genre names from dictionaries and return as a list
        return [genre['name'] for genre in genre_list]

    except json.JSONDecodeError as e:
        logging.error("Error decoding JSON string: {}".format(e))
    except KeyError as e:
        logging.error("Key 'name' not found in genre dictionary: {}".format(e))
    except Exception as e:
        logging.error("An error occurred while extracting genres: {}".format(e))
    return None

# 1. Load the dataset from a CSV file.

In [3]:
# Remove the restriction on the number of displayed columns
pd.options.display.max_columns = None

# Load the dataset
df = read_my_data(filepath="C:/Users/Mihail/Downloads/archive/movies_metadata.csv", low_memory=False)

In [4]:
# The shape is (number of rows, number of colums)
df.shape

(45466, 24)

In [5]:
# General info about the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [6]:
# Explore the first 5 rows
df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [7]:
# Column "id" contains dates that have to be removed
df = df[~df['id'].str.contains(r'\d{4}-\d{2}-\d{2}', na=False)]

In [8]:
# Here, column "id" is actually "tmdbId", so we rename it
df.rename(columns={'id': 'tmdbId'}, inplace=True)

In [9]:
# fix the data type of the column "tmdbId"
df["tmdbId"] = df["tmdbId"].astype('int64')

In [10]:
# reset the index
df = df.reset_index(drop=True)

In [11]:
# Won't use the other columns, so we'll build a smaller dataset
df_movies = df[ ['tmdbId', 'title', 'genres', 'release_date'] ]
df_movies.head(5)

,tmdbId,title,genres,release_date
0,862,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",1995-10-30
1,8844,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",1995-12-15
2,15602,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",1995-12-22
3,31357,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",1995-12-22
4,11862,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]",1995-02-10


In [12]:
# General info about the smaller dataset
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45463 entries, 0 to 45462
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tmdbId        45463 non-null  int64 
 1   title         45460 non-null  object
 2   genres        45463 non-null  object
 3   release_date  45376 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.4+ MB


In [13]:
# check for missing values
df_movies.isnull().sum()

tmdbId           0
title            3
genres           0
release_date    87
dtype: int64

In [14]:
# drop the rows with missing values
df_movies = df_movies.dropna(axis=0)

In [15]:
# check again
df_movies.isnull().sum()

tmdbId          0
title           0
genres          0
release_date    0
dtype: int64

In [16]:
# check again
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45376 entries, 0 to 45462
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tmdbId        45376 non-null  int64 
 1   title         45376 non-null  object
 2   genres        45376 non-null  object
 3   release_date  45376 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.7+ MB


# 2. Print the number of the unique movies in the dataset.

In [17]:
print("The number of unique movies is:", df_movies['title'].nunique())

The number of unique movies is: 42196


In [18]:
print("The number of unique movies is:", get_unique_values(df_movies, 'title'))

The number of unique movies is: 42196


# 3. Print the average rating of all the movies.

In [19]:
# Load the ratings dataset
df_ratings = read_my_data("C:/Users/Mihail/Downloads/archive/ratings_small.csv", low_memory=False)
df_ratings.head(5)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [20]:
# rows, columns
df_ratings.shape

(100004, 4)

In [21]:
# general info
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100004 non-null  int64  
 1   movieId    100004 non-null  int64  
 2   rating     100004 non-null  float64
 3   timestamp  100004 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [22]:
# check for missing values
df_ratings.isnull().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [23]:
# calculate the average rating
average_rating = np.mean(df_ratings['rating'])
print("The average ratings of all movies is:", round(average_rating, 2))

The average ratings of all movies is: 3.54


In [24]:
# using the function to calculate the average
calculate_average_value(df_ratings, 'rating')

3.54

# 4. Print the top 5 highest rated movies.

In [25]:
# Load the small links dataset.
df_links = read_my_data("C:/Users/Mihail/Downloads/archive/links_small.csv", low_memory=False)
df_links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [26]:
# numbers of rows and columns
df_links.shape

(9125, 3)

In [27]:
# general info
df_links.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9125 entries, 0 to 9124
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  9125 non-null   int64  
 1   imdbId   9125 non-null   int64  
 2   tmdbId   9112 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 214.0 KB


In [28]:
# check for missing values
df_links.isnull().sum()

movieId     0
imdbId      0
tmdbId     13
dtype: int64

In [29]:
# drop the missing values
df_links = df_links.dropna(axis=0)

In [30]:
# check again
df_links.isna().sum()

movieId    0
imdbId     0
tmdbId     0
dtype: int64

In [31]:
# fix the data type of the column "tmdbId"
df_links['tmdbId'] = df_links["tmdbId"].astype('int64')

In [32]:
# reset the index
df_links = df_links.reset_index(drop=True)

In [33]:
# Inner SQL Join
merged_df = merge_two_dataframes(df_ratings, df_links, how='inner', on='movieId')

In [34]:
merged_df.shape

(99933, 6)

In [35]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99933 entries, 0 to 99932
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   userId     99933 non-null  int64  
 1   movieId    99933 non-null  int64  
 2   rating     99933 non-null  float64
 3   timestamp  99933 non-null  int64  
 4   imdbId     99933 non-null  int64  
 5   tmdbId     99933 non-null  int64  
dtypes: float64(1), int64(5)
memory usage: 4.6 MB


In [36]:
# another SQL inner join with pandas
df_rated_movies = merge_two_dataframes(merged_df, df_movies, how='inner', on='tmdbId')

In [37]:
df_rated_movies.shape

(99850, 9)

In [38]:
df_rated_movies.head()

,userId,movieId,rating,timestamp,imdbId,tmdbId,title,genres,release_date
0,1,31,2.5,1260759144,112792,9909,Dangerous Minds,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",1995-08-11
1,7,31,3.0,851868750,112792,9909,Dangerous Minds,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",1995-08-11
2,31,31,4.0,1273541953,112792,9909,Dangerous Minds,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",1995-08-11
3,32,31,4.0,834828440,112792,9909,Dangerous Minds,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",1995-08-11
4,36,31,3.0,847057202,112792,9909,Dangerous Minds,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",1995-08-11


In [39]:
df_rated_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99850 entries, 0 to 99849
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   userId        99850 non-null  int64  
 1   movieId       99850 non-null  int64  
 2   rating        99850 non-null  float64
 3   timestamp     99850 non-null  int64  
 4   imdbId        99850 non-null  int64  
 5   tmdbId        99850 non-null  int64  
 6   title         99850 non-null  object 
 7   genres        99850 non-null  object 
 8   release_date  99850 non-null  object 
dtypes: float64(1), int64(5), object(3)
memory usage: 6.9+ MB


In [40]:
df_rated_movies.isnull().sum()

userId          0
movieId         0
rating          0
timestamp       0
imdbId          0
tmdbId          0
title           0
genres          0
release_date    0
dtype: int64

In [41]:
# group by 'title' and sort by average 'rating'
sorted_df_ratings = groupby_and_sort(df_rated_movies, 'title', 'rating')

In [42]:
sorted_df_ratings.head()

title
Female Perversions    5.0
Lake of Fire          5.0
Lamerica              5.0
The Family Stone      5.0
Riding Giants         5.0
Name: rating, dtype: float64

# 5. Print the number of movies released each year.

In [43]:
# Extract the year and assign it to the same column
df_rated_movies['release_date'] = (pd.to_datetime(df_rated_movies['release_date'])).dt.year

In [44]:
df_rated_movies.shape

(99850, 9)

In [45]:
df_rated_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99850 entries, 0 to 99849
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   userId        99850 non-null  int64  
 1   movieId       99850 non-null  int64  
 2   rating        99850 non-null  float64
 3   timestamp     99850 non-null  int64  
 4   imdbId        99850 non-null  int64  
 5   tmdbId        99850 non-null  int64  
 6   title         99850 non-null  object 
 7   genres        99850 non-null  object 
 8   release_date  99850 non-null  int32  
dtypes: float64(1), int32(1), int64(5), object(2)
memory usage: 6.5+ MB


In [46]:
df_rated_movies.head(3)

,userId,movieId,rating,timestamp,imdbId,tmdbId,title,genres,release_date
0,1,31,2.5,1260759144,112792,9909,Dangerous Minds,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",1995
1,7,31,3.0,851868750,112792,9909,Dangerous Minds,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",1995
2,31,31,4.0,1273541953,112792,9909,Dangerous Minds,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",1995


In [47]:
# group by 'release_date' and count 'title'
average_ratings = df_rated_movies.groupby('release_date')['title'].count()

In [48]:
average_ratings.sort_values(ascending=False)

release_date
1995    6562
1994    5911
1996    5074
1999    5070
1993    4180
        ... 
1918       2
1917       2
1915       2
1919       1
1916       1
Name: title, Length: 103, dtype: int64

# 6. Print the number of movies in each genre.

In [49]:
# Apply the function 'extract_genre_names' to 'genres' column 
df_rated_movies['genre_names'] = df_rated_movies['genres'].apply(extract_from_json)

# Explode the list of genre names to create multiple rows for each movie
# Transform each element of a list-like to a row, replicating index values.
df_exploded = df_rated_movies.explode('genre_names')

In [50]:
df_exploded.head(5)

,userId,movieId,rating,timestamp,imdbId,tmdbId,title,genres,release_date,genre_names
0,1,31,2.5,1260759144,112792,9909,Dangerous Minds,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",1995,Drama
0,1,31,2.5,1260759144,112792,9909,Dangerous Minds,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",1995,Crime
1,7,31,3.0,851868750,112792,9909,Dangerous Minds,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",1995,Drama
1,7,31,3.0,851868750,112792,9909,Dangerous Minds,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",1995,Crime
2,31,31,4.0,1273541953,112792,9909,Dangerous Minds,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",1995,Drama


In [51]:
# Movie genres: Counts
genre_counts = df_exploded.groupby('genre_names')["movieId"].count()
print(genre_counts)

genre_names
Action             25655
Adventure          21887
Animation           6123
Comedy             35893
Crime              17495
Documentary         1454
Drama              47067
Family             11868
Fantasy            12517
Foreign              202
History             3968
Horror              6140
Music               3915
Mystery             8803
Romance            18855
Science Fiction    15153
TV Movie              59
Thriller           25579
War                 3778
Western             1610
Name: movieId, dtype: int64


# 7. Save the dataset to a JSON file.

In [52]:
df_exploded.reset_index(inplace=True, drop=False)

In [53]:
df_exploded.head()

,index,userId,movieId,rating,timestamp,imdbId,tmdbId,title,genres,release_date,genre_names
0,0,1,31,2.5,1260759144,112792,9909,Dangerous Minds,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",1995,Drama
1,0,1,31,2.5,1260759144,112792,9909,Dangerous Minds,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",1995,Crime
2,1,7,31,3.0,851868750,112792,9909,Dangerous Minds,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",1995,Drama
3,1,7,31,3.0,851868750,112792,9909,Dangerous Minds,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",1995,Crime
4,2,31,31,4.0,1273541953,112792,9909,Dangerous Minds,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",1995,Drama


In [54]:
my_path = 'C:/Users/Mihail/Downloads/archive/dataset_exploded.json'
df_exploded.to_json(my_path, orient='index')

# 8. Upload the code in public GIT repository
    
Uploaded it to my public git repository - https://github.com/ArgentumZZ 

# 9. Provide setup.py or pyproject.toml

    movie/
    │    
    ├── __init__.py
    ├── setup.py
    ├── movie/
    │   ├── __init__.py
    │   ├── functions.py
    │   └── data_analysis.py
    │
    └── tests/
        ├── __init__.py
        └── unittests.py

# 10. Write unittests (optional)